In [2]:
import nltk
from nltk import word_tokenize
import numpy as np
import os
import codecs
positivesFiles = os.listdir('./books/Book/pos_Bk/')
negativesFiles = os.listdir('./books/Book/neg_Bk/')

#Loading Data
positives = list(map(lambda f: codecs.open('./books/Book/pos_Bk/'+f, 'r', encoding='latin1').read(), positivesFiles))
negatives = list(map(lambda f: codecs.open('./books/Book/neg_Bk/'+f, 'r', encoding='latin1').read(), negativesFiles))


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizerPos = CountVectorizer()

X = np.append(positives, negatives)
y = np.append(np.ones(len(positives)), np.zeros(len(negatives)))
Xtransorm = vectorizerPos.fit_transform(X)


stop = set(stopwords.words('english'))
vectorizerPos = TfidfVectorizer(stop_words = stop, tokenizer = word_tokenize, binary = True)
Xtransorm = vectorizerPos.fit_transform(X)

#TODO : tester une implémentation soit meme avec repeated k fold
clf = MultinomialNB()
cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
print('score on 10 Cross Val with 10 folds : ' + str(np.mean(cross_val_score(clf, Xtransorm, y, cv=cv))))  


score on training set : 0.964
score on 10 Cross Val with 10 folds : 0.7545000000000001


/home/arnaud/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  sorted(inconsistent))


score on 10 Cross Val with 10 folds : 0.7769999999999999


In [91]:
#Stemming / Lemmatisation
porter_stemmer = nltk.stem.porter.PorterStemmer()
wordnet_lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

#Tokenization of positives
tokenizedPos = list(map(lambda com: word_tokenize(com), positives))
stemsPos = np.array(tokenizedPos)
lemmesPos = np.array(tokenizedPos)

for i in range(len(positives)):
    tokensStem = np.array([])
    tokensLemm = np.array([])
    for word in tokenizedPos[i]:
        tokensStem = np.append(tokensStem, porter_stemmer.stem(word))
        tokensLemm = np.append(tokensLemm, wordnet_lemmatizer.lemmatize(word))
    stemsPos[i] = tokensStem
    lemmesPos[i] = tokensLemm
    

In [92]:
wordCount = {}


for comment in stemsPos:
    for stem in comment:
        try:
            wordCount[stem] = wordCount[stem] + 1
        except KeyError:
            wordCount[stem] = 1 

reductedStemPos = np.array(stemsPos)        

for i in range(len(reductedStemPos)):
    selectedStems = np.array([])
    for stem in reductedStemPos[i]:
        if (wordCount[stem] >= 1):
            selectedStems = np.append(selectedStems, stem)
    
    reductedStemPos[i] = selectedStems            

In [93]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = set(stopwords.words('english'))

positive_no_stopwords = list(positives)

for i in range(len(positives)):
    positive_no_stopwords[i] = [word for word in word_tokenize(positives[i].lower()) if word not in stop]


[nltk_data] Downloading package stopwords to /home/arnaud/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [121]:
from nltk import pos_tag

positives_no_closedClass = list(positives)

open_classes = {'FW', 'JJ', 'JRJ', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS',
                'RB', 'RBR', 'RBS', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}

for i in range(len(positives)):
    tokens = word_tokenize(positives_no_closedClass[i].lower())
    tags = pos_tag(tokens)
    positives_no_closedClass[i] = list(filter(lambda tagged_word: tagged_word[1] in open_classes,tags))
    positives_no_closedClass[i] = list(map(lambda x: x[0], positives_no_closedClass[i]))


In [123]:
#Word Presence #Normalement avec wordCount c'est bon
#tf-idf